<a href="https://www.kaggle.com/code/jatin2055/langchain-toolcalling?scriptVersionId=261608319" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [306]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [307]:
# pip install langchain langchain_core langchain_openai langchain_community

In [308]:
from langchain_core.tools import tool, InjectedToolArg
from typing import Annotated
import requests

# Tool Creation

In [309]:
# tool creation

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> dict:

    """
    This tool returns back the conversion factor required to 
    convert bas_currency to target_currency.
    """
    exchange_api_key = 'free_exchange_api'
    url = f"https://v6.exchangerate-api.com/v6/{exchange_api_key}/pair/{base_currency}/{target_currency}"

    response = requests.get(url)

    return response.json()
    

In [310]:
conversion_response = get_conversion_factor.invoke({"base_currency": "EUR", "target_currency": "INR"})

In [311]:
conversion_response

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1757721601,
 'time_last_update_utc': 'Sat, 13 Sep 2025 00:00:01 +0000',
 'time_next_update_unix': 1757808001,
 'time_next_update_utc': 'Sun, 14 Sep 2025 00:00:01 +0000',
 'base_code': 'EUR',
 'target_code': 'INR',
 'conversion_rate': 103.4962}

In [312]:
conversion_rate = conversion_response["conversion_rate"]

print(conversion_rate)

103.4962


In [313]:
# tool 2

@tool
def currency_convert(base_currency_value: float, conversion_factor: Annotated[float, InjectedToolArg]) -> float:
    """
    Provided the conversion factor, this tools converts base currency value to targeted currency
    """

    return base_currency_value*conversion_factor

In [314]:
targeted_currency_value = currency_convert.invoke({"base_currency_value": 10, "conversion_factor":conversion_rate})

In [315]:
targeted_currency_value

1034.962

# Tool Binding

In [316]:
from langchain_openai import ChatOpenAI



In [317]:
api_key = 'your_api_key'

In [359]:
llm = ChatOpenAI(api_key = api_key)

In [360]:
llm_with_tools = llm.bind_tools([get_conversion_factor, currency_convert])

#llm_with_tools = llm_with_tools.bind_tools([currency_convert])

# Memory management

In [330]:
from langchain_core.messages import HumanMessage, SystemMessage
import json

In [361]:
messages = [HumanMessage('Find the exchange rate between EUR and INR , based on the rate please convert CURRENCY 10 EUR to INR')] # What is the currency convertion rate between USD and INR and based on that rate

In [362]:
ai_message = llm_with_tools.invoke(messages)

In [357]:
messages.append(ai_message)

# Tool Execution

In [363]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'EUR', 'target_currency': 'INR'},
  'id': 'call_tXF9aIT0z2RJBxkIoXfS3ewe',
  'type': 'tool_call'}]

In [364]:
for tool in ai_message.tool_calls:

    if tool['name'] == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool)
        print(tool_message1)
        conversion_rate = json.loads(tool_message1.content)['conversion_rate']
        print(conversion_rate)
        messages.append(tool_message1)

    if tool['name'] == "currency_convert":
        tool['args']['conversion_factor'] = conversion_rate
        tool_message2 = currency_convert.invoke(tool)
        print(tool_message2)
        messages.append(tool_message1)
    

content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1757721601, "time_last_update_utc": "Sat, 13 Sep 2025 00:00:01 +0000", "time_next_update_unix": 1757808001, "time_next_update_utc": "Sun, 14 Sep 2025 00:00:01 +0000", "base_code": "EUR", "target_code": "INR", "conversion_rate": 103.4962}' name='get_conversion_factor' tool_call_id='call_tXF9aIT0z2RJBxkIoXfS3ewe'
103.4962


In [365]:
messages

[HumanMessage(content='Find the exchange rate between EUR and INR , based on the rate please convert CURRENCY 10 EUR to INR', additional_kwargs={}, response_metadata={}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1757721601, "time_last_update_utc": "Sat, 13 Sep 2025 00:00:01 +0000", "time_next_update_unix": 1757808001, "time_next_update_utc": "Sun, 14 Sep 2025 00:00:01 +0000", "base_code": "EUR", "target_code": "INR", "conversion_rate": 103.4962}', name='get_conversion_factor', tool_call_id='call_tXF9aIT0z2RJBxkIoXfS3ewe')]

In [366]:
query = [HumanMessage('Convert 10 Eur to INR')]
messages.append(query)

convert_message = llm_with_tools.invoke(query)

In [369]:
convert_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'EUR', 'target_currency': 'INR'},
  'id': 'call_OBmfys0TF2mUCR2spiR3Mq1m',
  'type': 'tool_call'}]

# Issue: Not able to get both the tools in the same query.